# 1. 커리큘럼 데이터 임베딩하여 db 만들기

In [15]:
import pandas as pd 

years = [2021, 2022, 2023, 2024, 2025]

df_인공지능학부 = list()
df_인공지능전공 = list()
df_소프트웨어전공 = list()

for year in years:
    df1 = pd.read_csv(f"{year} 인공지능학부 커리큘럼.csv")
    df_인공지능학부.append(df1)

    df2 = pd.read_csv(f"{year} 인공지능전공 커리큘럼.csv")
    df_인공지능전공.append(df2)

    df3 = pd.read_csv(f"{year} 소프트웨어전공 커리큘럼.csv")
    df_소프트웨어전공.append(df3)

In [16]:
df4 = pd.read_csv("2025 정보보안전공 커리큘럼.csv")
df5 = pd.read_csv("2024 정보보안전공 커리큘럼.csv")

df_정보보안전공 = [df4, df5]

In [17]:
def make_embedding_text(df):
    df['임베딩 텍스트'] = df['교과목명'] + " (" + df['학년'] + " " + df['학기'] + ", " + df['과목구분'] + ")"

In [51]:
df_curriculum = df_인공지능학부 + df_인공지능전공 + df_소프트웨어전공 + df_정보보안전공
len(df_curriculum)

17

In [19]:
for df in df_curriculum:
    make_embedding_text(df)

In [20]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

# 임베딩 생성
for i, df in enumerate(df_curriculum):
    embeddings = model.encode(df['임베딩 텍스트'].tolist(), convert_to_tensor=True)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings.cpu()))  # FAISS에 벡터 추가
    faiss.write_index(index, f"curriculum_faiss_{i+1}.index")


# 질문 받기

In [66]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd

# 🔹 Step 1: 사전 준비 (모델 및 인덱스 로드)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# 🔹 Step 2: FAISS 인덱스 파일 로드
index_files = [
    "curriculum_faiss_1.index", "curriculum_faiss_2.index", "curriculum_faiss_3.index",
    "curriculum_faiss_4.index", "curriculum_faiss_5.index", "curriculum_faiss_6.index",
    "curriculum_faiss_7.index", "curriculum_faiss_8.index", "curriculum_faiss_9.index",
    "curriculum_faiss_10.index", "curriculum_faiss_11.index", "curriculum_faiss_12.index",
    "curriculum_faiss_13.index", "curriculum_faiss_14.index", "curriculum_faiss_15.index",
    "curriculum_faiss_16.index", "curriculum_faiss_17.index"
]

indexes = [faiss.read_index(f) for f in index_files]  # 여러 개의 FAISS 인덱스 로드
index = faiss.IndexFlatL2(indexes[0].d)  # 새로운 통합 인덱스 생성
index = faiss.IndexIDMap(index)

In [67]:
# 🔹 Step 3: 원본 커리큘럼 데이터 로드 (문서 ID → 교과목 매칭)
df_files = [
    "2021 인공지능학부 커리큘럼.csv", "2022 인공지능학부 커리큘럼.csv", "2023 인공지능학부 커리큘럼.csv", "2024 인공지능학부 커리큘럼.csv", "2025 인공지능학부 커리큘럼.csv",
    "2021 인공지능전공 커리큘럼.csv", "2022 인공지능전공 커리큘럼.csv", "2023 인공지능전공 커리큘럼.csv", "2024 인공지능전공 커리큘럼.csv", "2025 인공지능전공 커리큘럼.csv",
    "2021 소프트웨어전공 커리큘럼.csv", "2022 소프트웨어전공 커리큘럼.csv", "2023 소프트웨어전공 커리큘럼.csv", "2024 소프트웨어전공 커리큘럼.csv", "2025 소프트웨어전공 커리큘럼.csv",
    "2024 정보보안전공 커리큘럼.csv", "2025 정보보안전공 커리큘럼.csv"
]
df_curriculum = [pd.read_csv(f) for f in df_files]

In [85]:
# 🔹 Step 4: 검색 함수 정의
def search_faiss(query: str, top_k: int = 5):
    """FAISS에서 가장 유사한 커리큘럼을 검색하여 반환"""
    query_vector = embedding_model.encode(query).astype(np.float32).reshape(1, -1)  # 쿼리 벡터 변환
    D, I = index.search(query_vector, top_k)  # FAISS에서 검색
    
    results = []
    for score, idx in zip(D[0], I[0]):
        results.append((idx, score))  # (문서 ID, 유사도 점수)
    
    return results

In [90]:
search_faiss("인공지능 교과목 수강 신청")

[(-1, 3.4028235e+38),
 (-1, 3.4028235e+38),
 (-1, 3.4028235e+38),
 (-1, 3.4028235e+38),
 (-1, 3.4028235e+38)]

In [41]:
# 🔹 Step 3: 임베딩 모델 로드
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# 🔹 Step 4: 검색 함수 정의
def search_faiss(query: str, top_k: int = 5):
    """FAISS 인덱스에서 가장 유사한 문서 검색"""
    query_vector = embedding_model.encode(query).astype(np.float32).reshape(1, -1)  # 쿼리 벡터 변환
    D, I = index.search(query_vector, top_k)  # FAISS에서 검색
    
    results = []
    for score, idx in zip(D[0], I[0]):
        if idx != -1:
            results.append((idx, score))  # (문서 ID, 유사도 점수)
    
    return results

In [59]:
import ollama 

# 🔹 Step 6: Ollama 기반 LLM 챗봇
def rag_ollama_chat(query: str, top_k: int = 5):
    """사용자의 질문을 FAISS로 검색하고 Ollama가 응답"""

    # 🔹 Step 6-1: FAISS에서 검색
    search_results = search_faiss(query, top_k)

    if not search_results:
        return "죄송합니다. 관련된 커리큘럼을 찾을 수 없습니다."

    # 🔹 Step 6-2: 검색된 문서 내용을 불러오기
    retrieved_docs = []
    for doc_id, score in search_results:
        for df in df_curriculum:
            if doc_id < len(df):  # 문서 ID가 유효한 경우
                subject_info = df.iloc[doc_id]  # 해당 교과목 정보 가져오기
                doc_text = (
                    f"- {subject_info['교과목명']} ({subject_info['학년']}학년 {subject_info['학기']}학기, {subject_info['과목구분']})"
                )
                retrieved_docs.append(doc_text)

    # 🔹 Step 6-3: LLM 프롬프트 생성
    retrieved_text = "\n".join(retrieved_docs)
    prompt = f"""
    🔍 다음 교과목 정보를 참고하여 사용자의 질문에 답변해 주세요:

    {retrieved_text}

    질문: {query}
    """

    # 🔹 Step 6-4: Ollama LLM 호출
    response = ollama.chat(model="tinyllama", messages=[{"role": "user", "content": prompt}])
    
    return response["message"]["content"]

In [60]:
# 🔹 Step 6: 챗봇 테스트
query_text = "인공지능학부 딥러닝 관련 과목을 알려줘"
response = rag_ollama_chat(query_text)

print("Ollama RAG 챗봇 응답:")
print(response)

Ollama RAG 챗봇 응답:
죄송합니다. 관련된 커리큘럼을 찾을 수 없습니다.
